In [ ]:
import os
import numpy as np
import pandas as pd
from scapy.all import sniff, IP, TCP, UDP
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import classification_report
import time
import threading
import iptc  # For managing iptables firewall

# Function to capture and process network packets
def packet_callback(packet):
    if IP in packet:
        ip_src = packet[IP].src
        ip_dst = packet[IP].dst
        if TCP in packet or UDP in packet:
            packet_data = {
                'src_ip': ip_src,
                'dst_ip': ip_dst,
                'protocol': packet.proto,
                'length': len(packet),
                'time': time.time()
            }
            network_data.append(packet_data)

# List to hold captured network traffic
network_data = []

# Packet sniffing in a separate thread
def start_sniffing():
    sniff(prn=packet_callback, store=0, timeout=60)  # Sniffing for 60 seconds

# Data Preprocessing
def preprocess_data(data):
    df = pd.DataFrame(data)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df['time_diff'] = df['time'].diff().fillna(0).dt.total_seconds()
    df['length'] = StandardScaler().fit_transform(df[['length']])
    return df

# Anomaly Detection with LSTM
def train_lstm_model(data):
    X = data[['length', 'time_diff']].values
    X = X.reshape((X.shape[0], 1, X.shape[1]))  # Reshaping for LSTM

    model = Sequential()
    model.add(LSTM(64, activation='relu', input_shape=(X.shape[1], X.shape[2])))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(X, np.ones(len(X)), epochs=5, batch_size=32, verbose=0)
    return model

# Anomaly Detection with Isolation Forest
def train_isolation_forest(data):
    X = data[['length', 'time_diff']].values
    model = IsolationForest(contamination=0.1)
    model.fit(X)
    return model

# Blocking suspicious IP using iptables
def block_ip(ip_address):
    os.system(f"iptables -A INPUT -s {ip_address} -j DROP")
    print(f"Blocked IP: {ip_address}")

# Real-time anomaly detection and firewall adaptation
def detect_and_block_anomalies():
    while True:
        if len(network_data) > 100:
            df = preprocess_data(network_data)
            model = train_isolation_forest(df)
            X = df[['length', 'time_diff']].values
            anomaly_flags = model.predict(X)

            for i, flag in enumerate(anomaly_flags):
                if flag == -1:
                    blocked_ip = network_data[i]['src_ip']
                    print(f"Anomaly detected from IP: {blocked_ip}")
                    block_ip(blocked_ip)
        time.sleep(30)

# Firewall Initialization and Threading
if __name__ == "__main__":
    # Start packet sniffing in a background thread
    sniff_thread = threading.Thread(target=start_sniffing)
    sniff_thread.start()

    # Start anomaly detection and firewall adaptation in a background thread
    detect_thread = threading.Thread(target=detect_and_block_anomalies)
    detect_thread.start()

    sniff_thread.join()
    detect_thread.join()